<a href="https://colab.research.google.com/github/ricardodeazambuja/Maple-Syrup-Pi-Camera/blob/main/Model_Maker_Object_Detection_Tutorial_modified_for_Maple_Syrup_Pi_Camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

######  Modified by [ricardodeazambuja.com](https://ricardodeazambuja.com)

# Object Detection with TensorFlow Lite Model Maker

In this colab notebook, you'll learn how to use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) library to train a custom object detection model capable of detecting ~~salads~~ RPI Zero W within images on ~~a mobile~~ any device that can run TFLite models.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.

You'll use the publicly available ~~*Salads*~~ really tiny RPI Zero W dataset, which was created from ~~the [Open Images Dataset V4](https://storage.googleapis.com/openimages/web/index.html)~~ [Google Image Search](https://www.google.com/imghp) annotated using [VGG VIA Annotator](https://www.robots.ox.ac.uk/~vgg/software/via/via.html).

Each image in the dataset contains objects labeled as one of the following classes:
* rpi0


The dataset contains the bounding-boxes specifying where each object locates, together with the object's label.

## Prerequisites


### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker==0.4.2 # I hope this will keep this notebook from breaking so often...
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0 # I hope this will keep this notebook from breaking so often...

We will need colab_utils to convert the dataset, augment and split into TRAIN, VALIDATION and TEST.

In [ ]:
!pip install -q git+https://github.com/ricardodeazambuja/colab_utils.git

Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tensorflow_examples.lite.model_maker.third_party.efficientdet.keras import train
from tensorflow_examples.lite.model_maker.third_party.efficientdet.keras import train_lib

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
from colab_utils import *

Fix some problems with the original implementation...



In [ ]:
from tensorflow_examples.lite.model_maker.third_party.efficientdet import inference
from tensorflow_examples.lite.model_maker.third_party.efficientdet.keras import postprocess

class DisplayCallback(tf.keras.callbacks.Callback):
  """Display inference result callback."""

  def __init__(self, sample_image, output_dir, update_freq=None):
    super().__init__()
    image_file = tf.io.read_file(sample_image)
    self.sample_image = tf.expand_dims(
        tf.image.decode_jpeg(image_file, channels=3), axis=0)
    self.update_freq = update_freq
    self.output_dir = output_dir
    self.step = 0

  def set_model(self, model: tf.keras.Model):
    self.model = model
    self.config = model.config
    self.width, self.height = self.config.image_size
    self.sample_image = tf.image.resize(self.sample_image,
                                        self.config.image_size,
                                        preserve_aspect_ratio=False)
    log_dir = os.path.join(self.config.model_dir, 'test_images')
    self.file_writer = tf.summary.create_file_writer(log_dir)
    self.min_score_thresh = self.config.nms_configs['score_thresh'] or 0.4
    self.max_boxes_to_draw = self.config.nms_configs['max_output_size'] or 100

  def on_train_batch_end(self, batch, logs=None):
    if self.update_freq and batch % self.update_freq == 0:
      self._draw_inference(batch)

  def _draw_inference(self, step):
    self.step += step
    num_classes = (self.config.num_classes-1)
    cls_outputs, box_outputs = self.model(self.sample_image, training=False)
    detections = postprocess.generate_detections(self.config,
                                                 cls_outputs,
                                                 box_outputs,
                                                 [1.0]*num_classes,
                                                 list(range(num_classes)))

    # detections => [id, x, y, w, h, score, class]
    ids, xmin, ymin, w, h, scores, classes = detections[0].numpy().swapaxes(1,0)
    # xmin /= self.width
    # ymin /= self.height
    # w /= self.width
    # h /= self.height
    boxes = np.vstack([xmin, ymin, w+xmin, h+ymin]).T
    image = inference.visualize_image(
        self.sample_image[0],
        boxes,
        classes.astype(int),
        scores,
        label_map=self.model.config.label_map,
        min_score_thresh=self.min_score_thresh,
        max_boxes_to_draw=self.max_boxes_to_draw)

    with self.file_writer.as_default():
      # tensorboard will do some weird normalization, but not if uint8
      # https://github.com/tensorflow/tensorboard/issues/1113
      tf.summary.image('Test image', tf.expand_dims(tf.cast(image, tf.uint8), axis=0), step=self.step)

    return boxes

The VGG VIA Annotator can be used directly from your browser by accessing the link below:  
https://www.robots.ox.ac.uk/~vgg/software/via/via.html


You can add more images by importing the json file downloaded below into VIA.

In [ ]:
# Tiny (really tiny!) annotated dataset in COCO json format (created using VGG Via Annotator)
# (no images, only links found using Google Image Search)
!wget -q https://gist.github.com/ricardodeazambuja/8bd4c073b4d9c5f85f62f069b15ec369/raw/800db96d60d9386f210525f05f91c86807874885/rpi_zero_from_google_images.json -O rpi_zero_from_google_images.json

### Prepare the dataset


The *RPI Zero W* dataset is available at:
 [`https://git.io/JcCrP`](https://gist.github.com/ricardodeazambuja/8bd4c073b4d9c5f85f62f069b15ec369/raw/800db96d60d9386f210525f05f91c86807874885/rpi_zero_from_google_images.json).

It contains... only 18 images for training! LoL.

Ideally, you should try to create a dataset as diverse as possible. The example in this notebook uses mostly images where the RPI0 lies on a desk, so it is far from diverse.

<br/>

The dataset is provided in COCO JSON format and we will convert it to [CSV format](https://cloud.google.com/vision/automl/object-detection/docs/csv-format):
```
VALIDATION,images/image_1.jpg,rpi0,0.16,0.17,,,0.70,0.69,,
TRAIN,images/image_3.jpg,rpi0,0.18,0.13,,,0.68,0.54,,
TEST,images/image_10.jpg,rpi0,0.18,0.22,,,0.71,0.56,,
```

* Each row corresponds to an object localized inside a larger image, with each object specifically designated as test, train, or validation data. You'll learn more about what that means in a later stage in this notebook.
* Each row has a label: `rpi0` (because we only annotated for that!)
* Bounding boxes are specified for each image using the top left and bottom right vertices.

## Quickstart

There are six steps to training an object detection model:

**Step 1. Choose an object detection model archiecture.**

This tutorial uses the EfficientDet-Lite0 model. EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Pixel 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>


In [ ]:
spec = model_spec.get('efficientdet_lite0')

In [ ]:
print(spec.config)

In [ ]:
spec.config['model_dir']

**Step 2. Load the dataset.**

Model Maker will take input data in the CSV format. Use the `object_detector.DataLoader.from_csv` method to load the dataset and split them into the training, validation and test images.

* Training images: These images are used to train the object detection model to recognize salad ingredients.
* Validation images: These are images that the model didn't see during the training process. You'll use them to decide when you should stop the training, to avoid [overfitting](https://en.wikipedia.org/wiki/Overfitting).
* Test images: These images are used to evaluate the final model performance.

You can load the CSV file directly from Google Cloud Storage, but you don't need to keep your images on Google Cloud to use Model Maker. You can specify a local CSV file on your computer, and Model Maker will work just fine.

In [ ]:
!rm -rf images

In [ ]:
cocojson2modelmakercsv("rpi_zero_from_google_images.json", "full_dataset.csv")

In [ ]:
!cat full_dataset.csv | wc -l

In [ ]:
!cat full_dataset.csv

In [ ]:
saveimgslocally("full_dataset.csv", "dataset_local.csv", img_path="images")

In [ ]:
# Check how many annotations were successful
!cat dataset_local.csv | wc -l

In [ ]:
!cat dataset_local.csv

In [ ]:
# We need to split it before the data augmentation or we will be most probably leaking test/validation to train...
splitdataset("dataset_local.csv", train_val_test_ratios=[0.8,0.1,0.1])

In [ ]:
!cat dataset_local.csv

In [ ]:
augment_dataset("dataset_local.csv", increaseby=3, seed=42)

In [ ]:
# Total number of annotations after the augmentation
!cat dataset_local.csv | wc -l

In [ ]:
!cat dataset_local.csv | grep TRAIN | wc -l
!cat dataset_local.csv | grep TEST | wc -l
!cat dataset_local.csv | grep VALIDATION | wc -l

In [ ]:
from PIL import Image

with open("dataset_local.csv", "r") as file:
  ds = file.read()

ann_number = 35 # 1 to 112

l = ds.splitlines()[ann_number-1].split(",")
print(l)
drawbox(Image.open(l[1]).resize((800,600)), [float(i) for i in [l[3],l[4], l[7], l[8]]])

In [ ]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('dataset_local.csv')

In [ ]:
train_data.label_map

In [ ]:
# %%bash -s "$test_data.annotations_json_file"
# cat $1

**Step 3. Train the TensorFlow model with the training data.**

* The EfficientDet-Lite0 model uses `epochs = 50` by default, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop early to avoid overfitting.
* Set `batch_size = 8` here so you will see that it takes 21 steps to go through the 175 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
 %load_ext tensorboard
#%reload_ext tensorboard

In [ ]:
!pkill tensorboard  # trust me, you will eventually need this...

In [ ]:
# from tensorboard import notebook
# notebook.list() # View open TensorBoard instances
#notebook.manager.start(["serve", "--logdir", f"{model_dir}"])  # starts a server using the model_dir
#notebook.display(port=6006, height=1000)  # just create the visualization, without starting a new server

In [ ]:
# If you want to set a custom dir:
# spec.config.model_dir = "your_custom_dir"
model_dir = spec.config.model_dir
model_dir

In [ ]:
ls $model_dir -la

In [ ]:
spec.config.sample_image = l[1]
spec.config.sample_image

In [ ]:
spec.config.img_summary_steps = 1  # step size (in relation to batches)
spec.config.img_summary_steps

In [ ]:
spec.config.map_freq = 1  # AP eval frequency in epochs.
spec.config.map_freq

In [ ]:
spec.config.verbose = 1 # shows when the checkpoints are really saved
spec.config.verbose

In [ ]:
# check https://github.com/tensorflow/examples/blob/37df0b9a2f7240831642cea4380e4f7ec33bee83/tensorflow_examples/lite/model_maker/third_party/efficientdet/aug/autoaugment.py
spec.config.autoaugment_policy = 'v2'  # None or v0 to v3, it will take longer to train it...
spec.config.autoaugment_policy

Create and setup the model and the datasets:

In [ ]:
batch_size = 8
with spec.ds_strategy.scope():
  object_detector_model = object_detector.create(train_data, model_spec=spec, batch_size=batch_size, train_whole_model=True, validation_data=validation_data, do_train=False)
  train_ds, steps_per_epoch, _ = object_detector_model._get_dataset_and_steps(
      train_data, batch_size, is_training=True)
  validation_ds, validation_steps, val_json_file = object_detector_model._get_dataset_and_steps(
      validation_data, batch_size, is_training=False)

  config = object_detector_model.model.config
  batch_size = config.batch_size
  config.update(
      dict(
          steps_per_epoch=steps_per_epoch,
          eval_samples=batch_size * validation_steps,
          val_json_file=val_json_file,
          batch_size=batch_size))
  train.setup_model(object_detector_model.model, config)
  train.init_experimental(config)

In [ ]:
print(config)

In [ ]:
object_detector_model.model.summary()

In [ ]:
%tensorboard --logdir $model_dir --bind_all

In [ ]:
# !tensorboard dev upload --logdir $model_dir \
#   --name "Model Maker Object Detection Tutorial - modified for Maple Syrup Pi Camera" \
#   --description "Training results from https://github.com/ricardodeazambuja/Maple-Syrup-Pi-Camera/blob/main/Model_Maker_Object_Detection_Tutorial_modified_for_Maple_Syrup_Pi_Camera.ipynb" \
#   --one_shot

Train the model for a certain number of epochs defined by `epochs`:     
(if you want to train more epochs, just update the `initial_epoch`, add it to epochs, and run the cell again)

In [ ]:
epochs = 50
initial_epoch = 0 # it's always (last_epoch-1), so it starts at 0
load_latest_checkpoint = False

if load_latest_checkpoint:
  object_detector_model.model.load_weights(tf.train.latest_checkpoint(model_dir))

with spec.ds_strategy.scope():
  train_lib.DisplayCallback = DisplayCallback # monkey patch with my alternative version...
  keras_cb = train_lib.get_callbacks(config.as_dict(), validation_ds)
  keras_cb[2].set_model(object_detector_model.model)
  keras_cb[3].set_model(object_detector_model.model)

  model = object_detector_model.model
  model.fit(
      train_ds,
      epochs=epochs,
      initial_epoch=initial_epoch,
      steps_per_epoch=steps_per_epoch,
      callbacks=keras_cb,
      validation_data=validation_ds,
      validation_steps=validation_steps)

In [ ]:
# to load a specific epoch (choose from the results above that best fit your needs, check tensorboard to have a better view)
# epoch = 10
# _ = object_detector_model.model.load_weights(model_dir + f"/ckpt-{epoch}")
# to load the latest
# _ = object_detector_model.model.load_weights(tf.train.latest_checkpoint(model_dir))

In [ ]:
# !rm -rf $model_dir/*  # this will delete all the saved stuff...

In [ ]:
# ls $model_dir -la

In [ ]:
# cat $model_dir/config.yaml

In [ ]:
# dir(object_detector_model.model)

**Step 4. Evaluate the model with the test data.**

After training the object detection model using the images in the training dataset, use the remaining 14 images in the test dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 steps to go through the 14 images in the test dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
object_detector_model.evaluate(train_data) # let's check how it performs on the training data... just to make sure it learned something

In [ ]:
object_detector_model.evaluate(test_data) # ok, now using the unseen test data!

In [ ]:
# model.create_serving_model?

**Step 5.  Export as a TensorFlow Lite model.**

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is full integer quantization.

In [ ]:
object_detector_model.export(export_dir='.')  # it takes some time (3min?)...

**Step 6.  Evaluate the TensorFlow Lite model.**

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
object_detector_model.evaluate_tflite('model.tflite', test_data) # the AP dropped a little bit after converting.

You can download the TensorFlow Lite model file using the left sidebar of Colab. Right-click on the `model.tflite` file and choose `Download` to download it to your local computer.

This model can be integrated into an Android or an iOS app using the [ObjectDetector API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) of the [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview).

See the [TFLite Object Detection sample app](https://github.com/tensorflow/examples/blob/master/lite/examples/object_detection/android/lib_task_api/src/main/java/org/tensorflow/lite/examples/detection/tflite/TFLiteObjectDetectionAPIModel.java#L91) for more details on how the model is used in an working app.

*Note: Android Studio Model Binding does not support object detection yet so please use the TensorFlow Lite Task Library.*

## (Optional) Test the TFLite model on your image

You can test the trained TFLite model using images from the internet.
* Replace the `INPUT_IMAGE_URL` below with your desired input image.
* Adjust the `DETECTION_THRESHOLD` to change the sensitivity of the model. A lower threshold means the model will pickup more objects but there will also be more false detection. Meanwhile, a higher threshold means the model will only pickup objects that it has confidently detected.

Although it requires some of boilerplate code to run the model in Python at this moment, integrating the model into a mobile app only requires a few lines of code.

In [ ]:
#@title Load the trained TFLite model and define some visualization functions

import cv2

from PIL import Image

model_path = 'model.tflite'

# Load the labels into a list
classes = ['???'] * object_detector_model.model_spec.config.num_classes
label_map = object_detector_model.model_spec.config.label_map
for label_id, label_name in label_map.as_dict().items():
  classes[label_id-1] = label_name

# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  return resized_img, original_image


def set_input_tensor(interpreter, image):
  """Set the input tensor."""
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image


def get_output_tensor(interpreter, index):
  """Retur the output tensor at the given index."""
  output_details = interpreter.get_output_details()[index]
  tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
  return tensor


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  # # Feed the input image to the model
  # set_input_tensor(interpreter, image)
  # interpreter.invoke()
  # # Get all outputs from the model
  # scores = get_output_tensor(interpreter, 0)
  # boxes = get_output_tensor(interpreter, 1)
  # count = int(get_output_tensor(interpreter, 2))
  # classes = get_output_tensor(interpreter, 3)

  signature_fn = interpreter.get_signature_runner()
  # Feed the input image to the model
  output = signature_fn(images=image.numpy().astype('uint8'))
  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
# https://cdn10.bigcommerce.com/s-u7jmw/products/128/images/649/pi-zero-hub-hat-v1.3-04__51773.1468825053.1280.1280.jpg?c=2
# https://cdn11.bigcommerce.com/s-am5zt8xfow/images/stencil/1280x1280/products/1545/4666/Clster_HAT_v2_4_of_4_512x5121__94693.1540085018__28928.1584561811.jpg?c=2

In [ ]:
#@title Run object detection and show the detection results on unseen image

INPUT_IMAGE_URL = "https://cdn11.bigcommerce.com/s-am5zt8xfow/images/stencil/1280x1280/products/1545/4666/Clster_HAT_v2_4_of_4_512x5121__94693.1540085018__28928.1584561811.jpg?c=2" #@param {type:"string"}
DETECTION_THRESHOLD = 0.5 #@param {type:"number"}

TEMP_FILE = '/tmp/image.png'

!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
im = Image.open(TEMP_FILE)
im.thumbnail((512, 512), Image.ANTIALIAS)
im.save(TEMP_FILE, 'PNG')

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    TEMP_FILE,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
Image.fromarray(detection_result_image)

## (Optional) Compile For the Edge TPU

Now that you have a quantized EfficientDet Lite model, it is possible to compile and deploy to a [Coral EdgeTPU](https://coral.ai/).

**Step 1. Install the EdgeTPU Compiler**

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

**Step 2. Select number of Edge TPUs, Compile**

The EdgeTPU has 8MB of SRAM for caching model paramaters ([more info](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching)). This means that for models that are larger than 8MB, inference time will be increased in order to transfer over model paramaters. One way to avoid this is [Model Pipelining](https://coral.ai/docs/edgetpu/pipeline/) - splitting the model into segments that can have a dedicated EdgeTPU. This can significantly improve latency.

The below table can be used as a reference for the number of Edge TPUs to use - the larger models will not compile for a single TPU as the intermediate tensors can't fit in on-chip memory.

| Model architecture | Minimum TPUs | Recommended TPUs
|--------------------|-------|-------|
| EfficientDet-Lite0 | 1     | 1     |
| EfficientDet-Lite1 | 1     | 1     |
| EfficientDet-Lite2 | 1     | 2     |
| EfficientDet-Lite3 | 2     | 2     |
| EfficientDet-Lite4 | 2     | 3     |

**NEW:** The edgetpu runtime version installed in the RPI image is 13, therefore you need to pass `-m 13` to the edgetpu_compiler  

In [ ]:
NUMBER_OF_TPUS =  1#@param {type:"number"}
RUNTIME_VERSION =  13#@param {type:"number"}
!edgetpu_compiler  -m=$RUNTIME_VERSION model.tflite --num_segments=$NUMBER_OF_TPUS

**Step 3. Download, Run Model**

With the model(s) compiled, they can now be run on EdgeTPU(s) for object detection. First, download the compiled TensorFlow Lite model file using the left sidebar of Colab. Right-click on the `model_edgetpu.tflite` file and choose `Download` to download it to your local computer.

Now you can run the model in your preferred manner. Examples of detection include:
* [pycoral detection](https://github.com/google-coral/pycoral/blob/master/examples/detect_image.py)
* [Basic TFLite detection](https://github.com/google-coral/tflite/tree/master/python/examples/detection)
* [Example Video Detection](https://github.com/google-coral/examples-camera)
* [libcoral C++ API](https://github.com/google-coral/libcoral)


## Advanced Usage

This section covers advanced usage topics like adjusting the model and the training hyperparameters.

### Load the dataset

#### Load your own data

You can upload your own dataset to work through this tutorial. Upload your dataset by using the left sidebar in Colab.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_object_detection.png" alt="Upload File" width="1000" hspace="0">

If you prefer not to upload your dataset to the cloud, you can also locally run the library by following the [guide](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

#### Load your data with a different data format

The Model Maker library also supports the `object_detector.DataLoader.from_pascal_voc` method to load data with [PASCAL VOC](https://towardsdatascience.com/coco-data-format-for-object-detection-a4c5eaf518c5#:~:text=Pascal%20VOC%20is%20an%20XML,for%20training%2C%20testing%20and%20validation) format. [makesense.ai](https://www.makesense.ai/) and [LabelImg](https://github.com/tzutalin/labelImg) are the tools that can annotate the image and save annotations as XML files in PASCAL VOC data format:
```python
object_detector.DataLoader.from_pascal_voc(image_dir, annotations_dir, label_map={1: "person", 2: "notperson"})
```


### Customize the EfficientDet model hyperparameters

The model and training pipline parameters you can adjust are:

* `model_dir`: The location to save the model checkpoint files. If not set, a temporary directory will be used.
* `steps_per_execution`: Number of steps per training execution.
* `moving_average_decay`: Float. The decay to use for maintaining moving averages of the trained parameters.
* `var_freeze_expr`: The regular expression to map the prefix name of variables to be frozen which means remaining the same during training. More specific, use `re.match(var_freeze_expr, variable_name)` in the codebase to map the variables to be frozen.
* `tflite_max_detections`: integer, 25 by default. The max number of output detections in the TFLite model.
* `strategy`:  A string specifying which distribution strategy to use. Accepted values are 'tpu', 'gpus', None. tpu' means to use TPUStrategy. 'gpus' mean to use MirroredStrategy for multi-gpus. If None, use TF default with OneDeviceStrategy.
* `tpu`:  The Cloud TPU to use for training. This should be either the name used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 url.
* `use_xla`: Use XLA even if strategy is not tpu. If strategy is tpu, always use XLA, and this flag has no effect.
* `profile`: Enable profile mode.
* `debug`: Enable debug mode.

Other parameters that can be adjusted is shown in [hparams_config.py](https://github.com/google/automl/blob/df451765d467c5ed78bbdfd632810bc1014b123e/efficientdet/hparams_config.py#L170).


For instance, you can set the `var_freeze_expr='efficientnet'` which freezes the variables with name prefix `efficientnet` (default is `'(efficientnet|fpn_cells|resample_p6)'`). This allows the model to freeze untrainable variables and keep their value the same through training.

```python
spec = model_spec.get('efficientdet-lite0')
spec.config.var_freeze_expr = 'efficientnet'
```

### Change the Model Architecture

You can change the model architecture by changing the `model_spec`. For instance, change the `model_spec` to the EfficientDet-Lite4 model.

```python
spec = model_spec.get('efficientdet-lite4')
```

### Tune the training hyperparameters

The `create` function is the driver function that the Model Maker library uses to create models. The `model_spec` parameter defines the model specification. The `object_detector.EfficientDetSpec` class is currently supported. The `create` function comprises of the following steps:

1. Creates the model for the object detection according to `model_spec`.
2. Trains the model.  The default epochs and the default batch size are set by the `epochs` and `batch_size` variables in the `model_spec` object.
You can also tune the training hyperparameters like `epochs` and `batch_size` that affect the model accuracy. For instance,

*   `epochs`: Integer, 50 by default. More epochs could achieve better accuracy, but may lead to overfitting.
*   `batch_size`: Integer, 64 by default. The number of samples to use in one training step.
*   `train_whole_model`: Boolean, False by default. If true, train the whole model. Otherwise, only train the layers that do not match `var_freeze_expr`.

For example, you can train with less epochs and only the head layer. You can increase the number of epochs for better results.

```python
model = object_detector.create(train_data, model_spec=spec, epochs=10, validation_data=validation_data)
```

### Export to different formats

The export formats can be one or a list of the following:

*   `ExportFormat.TFLITE`
*   `ExportFormat.LABEL`
*   `ExportFormat.SAVED_MODEL`




By default, it exports only the TensorFlow Lite model file containing the model [metadata](https://www.tensorflow.org/lite/convert/metadata) so that you can later use in an on-device ML application. The label file is embedded in metadata.

In many on-device ML application, the model size is an important factor. Therefore, it is recommended that you quantize the model to make it smaller and potentially run faster. As for EfficientDet-Lite models, full integer quantization  is used to quantize the model by default. Please refer to [Post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) for more detail.

```python
model.export(export_dir='.')
```

You can also choose to export other files related to the model for better examination. For instance, exporting both the saved model and the label file as follows:
```python
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])
```

### Customize Post-training quantization on the TensorFlow Lite model

[Post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) is a conversion technique that can reduce model size and inference latency, while also improving CPU and hardware accelerator inference speed, with a little degradation in model accuracy. Thus, it's widely used to optimize the model.

Model Maker library applies a default post-training quantization techique when exporting the model. If you want to customize post-training quantization, Model Maker supports multiple post-training quantization options using [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig) as well. Let's take float16 quantization as an instance. First, define the quantization config.

```python
config = QuantizationConfig.for_float16()
```


Then we export the TensorFlow Lite model with such configuration.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# Read more

You can read our [object detection](https://www.tensorflow.org/lite/examples/object_detection/overview) example to learn technical details. For more information, please refer to:

*   TensorFlow Lite Model Maker [guide](https://www.tensorflow.org/lite/guide/model_maker) and [API reference](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker).
*   Task Library: [ObjectDetector](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) for deployment.
*   The end-to-end reference apps: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/ios), and [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/raspberry_pi).

